In [ ]:
# 测试1： GPT-4V直接读文档
from openai import OpenAI
import json
client = OpenAI()
SampleJson = '''{
  "load_customer": {
    "customer_company_name": "C.H. Robinson",
    "contact_person_first_name": "",
    "contact_person_last_name": "",
    "contact_email": "LoadDocs@chrobinson.com",
    "contact_number": "866-581-0813."
  },
  "equipment_type": "Van",
  "equipment_length": "53",
  "reference": "402493667",
  "weight": 40000,
  "rate": 900,
  "notes": "Fax POD to 866-746-1562 immediately upon delivery! Thanks!",
  "pickup_info": {
    "business_name": "Bulloch Fertilizer Co",
    "contact_number": "912-764-9084",
    "location_name": "STATESBORO, GA 30458",
    "location_street": "15 Blitch Lane",
    "time_range": {
      "start_time": 1656403200000,
      "end_time": 1656428400000
    },
    "site_instruction": ""
  },
  "dropoff_info": {
    "business_name": "Helena INDUSTRIES",
    "contact_number": "770-945-0686",
    "location_name": "Suwanee, GA 30024",
    "location_street": "3211 Shawnee Industrial",
    "time_range": {
      "start_time":  1656489600000,
      "end_time": 1656518400000
    },
    "site_instruction": ""
  }
}'''
serialized_json = json.dumps(SampleJson)
def extract():
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "system",
                "content":"You are a Json extractor for a US fleet company. You will receive Rate Confirmation from broker, and your job is extract info from image to the following Json format. When you can't find an answer, just put null, don't make things up. Also don't generate anything other than json."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"this is an exapmle, the json extracted is: {serialized_json}"},
                    {
                        "type": "image_url", "image_url": {"url":"https://raw.githubusercontent.com/Andrew-ppppeng/Explore-LLM-in-COMMAND/main/1-1.png"},
                    },
                    {
                        "type": "image_url", "image_url": {"url": "https://raw.githubusercontent.com/Andrew-ppppeng/Explore-LLM-in-COMMAND/main/1-2.png"},
                    },
                ],
            },
            {
                "role": "user",
                "content":[
                    {
                        "type":"text",
                        "text": "extract info in this img to json, follow the format above"
                    },
                    {
                        "type": "image_url",
                        # 可以设置detail：high来上传高精度图片。如果不设置，OPENAI基于图片自动判断。经测试，RC图片精度自动为high
                        "image_url": {"url": "https://raw.githubusercontent.com/Andrew-ppppeng/Explore-LLM-in-COMMAND/main/2-1.jpg","detail":"high"}
                    },
                    {
                        "type": "image_url", "image_url": {"url":"https://raw.githubusercontent.com/Andrew-ppppeng/Explore-LLM-in-COMMAND/main/2-2.jpg"}
                    }
                ]
            }
        ],

        max_tokens=1000,
    )
    return response

R = extract()

print(R)

In [ ]:
# 以下为运行结果。   
ChatCompletion(
    id='chatcmpl-8L5kYPLKezc1YzYXHn4VYdUReRDRs', 
    choices=[
        Choice(
            finish_reason=None, 
            index=0, 
            message=ChatCompletionMessage(
                content='```json\n{\n  "load_customer": {\n    "customer_company_name": "C.H. Robinson",\n    "contact_person_first_name": "Heather",\n    "contact_person_last_name": "Ramirez",\n    "contact_email": "null",\n    "contact_number": "888-278-9441."\n  },\n  "equipment_type": "Van",\n  "equipment_length": "53",\n  "reference": "411928026",\n  "weight": 43220,\n  "rate": 1500,\n  "notes": "SHIPPER - REFRESCO: Interstate 90 East or West, to I-95 (Dunkirk/Fredonia) pay toll and T/R at first stop light onto 60 North. Go 2 miles and at the third light T/R onto Cliffstar Ave, go 2 blks into Cliffstar.  DRIVER MUST NOT BREAK SEAL RECEIVER WILL REJECT",\n  "pickup_info": {\n    "business_name": "REFRESCO",\n    "contact_number": "716-363-3277",\n    "location_name": "DUNKIRK, NY 140482899",\n    "location_street": "1 Cliffstar Ave",\n    "time_range": {\n      "start_time": 1664334000000,\n      "end_time": 1664352000000\n    },\n    "site_instruction": "null"\n  },\n  "dropoff_info": {\n    "business_name": "Community FoodBank of New Jersey",\n    "contact_number": "908-355-3663",\n    "location_name": "HILLSIDE, NJ 07205",\n    "location_street": "31 Evans Terminal",\n    "time_range": {\n      "start_time": 1664420400000,\n      "end_time": 1664438400000\n    },\n    "site_instruction": "null"\n  }\n}\n```', 
                role='assistant', 
                function_call=None, 
                tool_calls=None), 
            finish_details={'type': 'stop', 'stop': '<|fim_suffix|>'}
        )
    ], 
    created=1700037682, 
    model='gpt-4-1106-vision-preview', 
    object='chat.completion', 
    system_fingerprint=None, 
    usage=CompletionUsage(completion_tokens=390, prompt_tokens=4298, total_tokens=4688))
# 390/1000 * 0.03 + 4298/1000 * 0.01 = $0.05468
# 最近三个月，平均一个月330次调用，花费 330*$0.05468 = $18.04